In [21]:
import os
import json
import sys
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
sys.path.append('../../src')
import DISData as DD
import time

In [22]:
doUT = DD.SQL()

host:  192.168.0.201
user:  diadmin
password:  ········


SQL connection done


In [23]:
query1_1 = (f'''CREATE TABLE IF NOT EXISTS preventing_drowsy_controlled_box_info(
            id INT PRIMARY KEY AUTO_INCREMENT,
            img_name VARCHAR(255) NOT NULL,
            img_dir VARCHAR(255) NOT NULL,
            img_format VARCHAR(10) NOT NULL,
            img_width INT NOT NULL,
            img_height INT NOT NULL,
            color_space VARCHAR(10) NOT NULL,
            label_name VARCHAR(255) NOT NULL,
            label_dir VARCHAR(1000) NOT NULL,
            label_format VARCHAR(10) NOT NULL,
            occupant_id VARCHAR(10) NOT NULL,
            occupant_sex VARCHAR(20) NOT NULL,
            occupant_age VARCHAR(20) NOT NULL,
            accessory VARCHAR(20),
            box_label VARCHAR(50) NOT NULL,
            box_xtl INT,
            box_ytl INT,
            box_xbr INT,
            box_ybr INT,
            box_rate INT);''')

In [24]:
query2_1 = (f'''INSERT INTO preventing_drowsy_controlled_box_info 
        (img_name, img_dir, img_format, img_width, img_height, color_space,
        label_name, label_dir, label_format,
        occupant_id, occupant_sex, occupant_age, accessory, box_label, box_xtl, box_ytl, box_xbr, box_ybr, box_rate)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)''')

In [25]:
try:
    doUT.create_table(query1_1)
except Exception as e:
    print(f"Error during table creation: {e}")

create table done


In [26]:
labels = {1:'M', 2:'F'}

In [27]:
project_id = None
value_list1 = []

time1 = time.time()
print(f'start: {time1}')

json_dir = "Z:\\preventing_drowsy_controlled\\data\\IR\\labeldata\\"
label_ids = os.listdir(json_dir)
for num, label_id in enumerate(label_ids):
    label_dir = f'{json_dir}\\{label_id}'
    label_lists = os.listdir(label_dir)
    color_space = 'IR'
    for label_file in label_lists:
        label_path = f'{json_dir}\\{label_id}\\{label_file}'
        with open(label_path,'r',encoding='utf-8') as f:
            config = json.load(f)

        img_name = config['FileInfo']['FileName']
        img_dir = '/'.join(os.path.dirname(label_file).replace('label','raw').split('\\')[-6:]+[f'{img_name}'])
        img_format = img_name.split('.')[-1]

        label_dir = '/'.join(os.path.dirname(label_file).split('\\')[-6:]+[os.path.basename(label_file)])

        label_name, label_format = os.path.splitext(os.path.basename(label_file))
        label_format = label_format.replace('.','')

        img_height = config['FileInfo']['Height']
        img_width = config['FileInfo']['Width']
        img_area = int(img_height) * int(img_width)
        # print(config)

        occupant_id = config['UserInfo']['ID']
        occupant_sex = labels[config['UserInfo']['Gender']]
        occupant_age = config['UserInfo']['Age']

        bbox = config['ObjectInfo']['BoundingBox']
        accessories = config['Accessory']

        accessory = []



        for i in accessories:
            if accessories[i] == True:
                accessory.append(i)
        accessory = ",".join(map(str,accessory))

        if accessory == []:
            accessory = 'NULL'

        for box_obj in bbox:
            time3 = time.time()
            # print(f'start_{label_file}: {time3}')
            box = bbox[box_obj]
            if box['isVisible'] == True:
                if not 'Opened' in box:
                    box_label = box_obj
                elif box['Opened'] == True:
                    box_label = f'{box_obj}_Open'
                elif box['Opened'] == False: 
                    box_label = f'{box_obj}_Close'
                box_xtl, box_ytl, box_xbr, box_ybr = box['Position']
                box_area = abs(int(box_xtl) - int(box_xbr)) * abs(int(box_ytl) - int(box_ybr))

                box_rate = round((box_area / img_area * 100),2)
                img_dir = img_dir.replace('Z:','DataBase')
                label_dir = label_dir.replace('Z:','DataBase')

                values1 = (img_name, img_dir, img_format, img_width, img_height, color_space,
                          label_name, label_dir, label_format,
                          occupant_id, occupant_sex, occupant_age, accessory, box_label, box_xtl, box_ytl, box_xbr, box_ybr, box_rate)
                # print(values1)
                value_list1.append(values1)

                if len(value_list1) >= 500:
                    doUT.insert_dataset_values(query2_1, value_list1)
                    value_list1 = []
                    print(f'{num} / {len(label_ids)}')

doUT.insert_dataset_values(query2_1, value_list1)
# print(value_list2)
value_list1 = []
print('done')    

start: 1718688395.8431437
insert 500 done
0 / 243
insert 500 done
0 / 243
insert 500 done
0 / 243
insert 500 done
1 / 243
insert 500 done
1 / 243
insert 500 done
1 / 243
insert 500 done
1 / 243
insert 500 done
2 / 243
insert 500 done
2 / 243
insert 500 done
2 / 243
insert 500 done
2 / 243
insert 500 done
3 / 243
insert 500 done
3 / 243
insert 500 done
3 / 243
insert 500 done
4 / 243
insert 500 done
4 / 243
insert 500 done
4 / 243
insert 500 done
4 / 243
insert 500 done
5 / 243
insert 500 done
5 / 243
insert 500 done
5 / 243
insert 500 done
6 / 243
insert 500 done
6 / 243
insert 500 done
6 / 243
insert 500 done
6 / 243
insert 500 done
7 / 243
insert 500 done
7 / 243
insert 500 done
7 / 243
insert 500 done
7 / 243
insert 500 done
8 / 243
insert 500 done
8 / 243
insert 500 done
8 / 243
insert 500 done
9 / 243
insert 500 done
9 / 243
insert 500 done
9 / 243
insert 500 done
9 / 243
insert 500 done
10 / 243
insert 500 done
10 / 243
insert 500 done
10 / 243
insert 500 done
10 / 243
insert 500